In [6]:
import openslide
import openslide.deepzoom
import os
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import shutil
from tqdm import tqdm

In [7]:
img = openslide.OpenSlide('eaae490bfc1d9f10fd34caf902a6900d_mask.tiff')

In [8]:
img.properties

<_PropertyMap {'openslide.level-count': '3', 'openslide.level[0].downsample': '1', 'openslide.level[0].height': '18432', 'openslide.level[0].tile-height': '512', 'openslide.level[0].tile-width': '512', 'openslide.level[0].width': '36864', 'openslide.level[1].downsample': '4', 'openslide.level[1].height': '4608', 'openslide.level[1].tile-height': '512', 'openslide.level[1].tile-width': '512', 'openslide.level[1].width': '9216', 'openslide.level[2].downsample': '16', 'openslide.level[2].height': '1152', 'openslide.level[2].tile-height': '512', 'openslide.level[2].tile-width': '512', 'openslide.level[2].width': '2304', 'openslide.quickhash-1': '53bd2260e5b30592a76c43a5a5ace425aa363b8c9b4197bba0d0f86db68c9db4', 'openslide.vendor': 'generic-tiff', 'tiff.ResolutionUnit': 'centimeter', 'tiff.XResolution': '20833.376803228592', 'tiff.YResolution': '20833.376803228592'}>

In [9]:
# APIs: https://openslide.org/api/python/#example-programs
# useful video: https://www.youtube.com/watch?v=QntLBvUZR5c
print(f"meta information:")
print(f"format: {img.detect_format('tumor_109.tif')}")
print(f"level_count: {img.level_count}")
print(f"dimensions: {img.dimensions}")
print(f"level_dimensions: {img.level_dimensions}")
print(f"level_downsamples: {img.level_downsamples}")


meta information:
format: philips
level_count: 3
dimensions: (36864, 18432)
level_dimensions: ((36864, 18432), (9216, 4608), (2304, 1152))
level_downsamples: (1.0, 4.0, 16.0)


In [13]:
slide_thumb_600 = img.get_thumbnail(size=(1600,1600))
slide_thumb_600.show()

In [17]:
dz = openslide.deepzoom.DeepZoomGenerator(img, tile_size = 512, overlap = 1, limit_bounds=False)

In [72]:
'''
Given a medical slide image, convert it to a tilesets using deepzoom
Image saving properties: https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#jpeg
@filename - the file name of the slide image, 
@tile_size: (int) – the width and height of a single tile. For best viewer performance, tile_size + 2 * overlap should be a power of two.
@tile_overlap (int) – the number of extra pixels to add to each interior edge of a tile
limit_bounds (bool) – True to render only the non-empty slide region
'''
def generateTile(filename, tile_size, tile_overlap, tile_format):
    dir = os.path.splitext(filename)[0]
    if(os.path.isdir(dir)):
        shutil.rmtree(dir)
        os.mkdir(dir)
    else:
        os.mkdir(dir)
    slide = openslide.OpenSlide(filename)
    tiles = openslide.deepzoom.DeepZoomGenerator(slide, tile_size = tile_size, overlap = tile_overlap, limit_bounds=False)
    text_file = open(f"{dir}/{dir}.dzi", "w")
    text_file.write(tiles.get_dzi(tile_format))
    text_file.close()
    dir = f"{dir}/{dir}_files"
    os.mkdir(dir)
    print(f"number of levels: {tiles.level_count}")
    print(f"number of tiles: {tiles.tile_count}")
    tile_count = tiles.tile_count
    level_count = tiles.level_count
    for level in tqdm(range(level_count)):
        level_dir = f"{dir}/{level}"
        os.mkdir(level_dir)
        cols, rows = tiles.level_tiles[level]
        for row in range(rows):
            for col in range(cols):
                tile_name = f"{dir}/{level}/{col}_{row}.{tile_format}"
                temp_tile = tiles.get_tile(level, (col, row))
                temp_tile.save(tile_name, 'JPEG')
                      
generateTile('tumor_109.tif', 512, 1, 'jpeg')


number of levels: 18
number of tiles: 59834


100%|██████████| 18/18 [16:54<00:00, 56.37s/it] 


# get ground truth data

In [5]:
import json
import xmltodict

In [8]:

with open('2bb4f3202acd0b3cd10ffae292b6edf8_mask.xml', 'r', encoding='utf-8') as file:
    my_xml = file.read()

gt = xmltodict.parse(my_xml)
  

In [9]:
gt_data = gt['ASAP_Annotations']['Annotations']['Annotation']
jsonData = {'data':[]}
for annotation in gt_data:
    #if str(annotation['@PartOfGroup']) != "_2":
    record = {}
    record['@context'] = "http://www.w3.org/ns/anno.jsonld"
    record['id'] = annotation['@Name']
    record['type'] = 'Annotation'
    record['body'] = {}
    if str(annotation['@PartOfGroup']) == "_0": 
        if str(annotation['@PartOfGroup']) == "_0":
            record['body']['name'] = "Tumor"
            record['body']['value'] = "lab_2"
        if str(annotation['@PartOfGroup']) == "_2":
            print(str(annotation['@PartOfGroup']))
            record['body']['name'] = "Benign"
            record['body']['value'] = "lab_3"
        record['body']['purpose'] = 'tagging'
        record['body']['type'] = 'TextualBody' 
        record['target'] = {}
        record['target']['source'] = "https://camelyon17.grand-challenge.org/Data/"
        record['target']['selector'] = {}
        record['target']['selector']['type'] = 'SvgSelector'
        #"<svg><polygon points="34761.37109375,13919.111328125 58194.9453125,14564.1015625 50868.2890625,22088.78125 31129.6015625,21207.041015625 31162.7421875,21148.982421875"></polygon></svg>"
        coords = ""
        
        if type(annotation['Coordinates']['Coordinate']) is dict:
            print(annotation['Coordinates']['Coordinate'])
        else:
            for value in annotation['Coordinates']['Coordinate']:
                #print(value)
                coords += value['@X'] + ',' + value['@Y'] + ' '
            record['target']['selector']['value'] = "<svg><polygon points=\""+coords+"\"></polygon></svg>"
            jsonData['data'].append(record)

with open('pandasneg.json', 'w') as fp:
    json.dump(jsonData, fp)

#jsonData

TypeError: 'NoneType' object is not subscriptable